In [225]:
import ee
import geemap
import geopandas as gpd

In [226]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-clowd')

In [227]:
shape_name="Area"
scale = 100
n_tree = 500

In [228]:
# Path to the shapefile in Google Drive
shapefile_path = f'shape-file/{shape_name}/{shape_name}.shp'

# Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Convert the GeoPandas DataFrame to a GeoJSON
gdf = gdf.to_crs("EPSG:4326")  # Make sure the CRS is WGS84
geojson = gdf.__geo_interface__

# Convert GeoJSON to an Earth Engine FeatureCollection
geometry = ee.FeatureCollection(geojson)

In [229]:
# Fetch GLDAS data
GLDAS = ee.ImageCollection("NASA/GLDAS/V022/CLSM/G025/DA1D") \
    .filterDate('2023-01-01', '2024-01-01') \
    .filterBounds(geometry) \
    .select('GWS_tavg') \
    .mean() \
    .clip(geometry)

# Visualization parameters
colorizedVis = {
    'min': 900,
    'max': 3000,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ],
}

In [230]:
# Fetch Sentinel-1 data
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterDate('2023-01-01', '2024-01-01') \
    .filterBounds(geometry) \
    .mean() \
    .clip(geometry)

# Fetch SRTM data
SRTM = ee.Image("USGS/SRTMGL1_003").clip(geometry)

In [231]:
# Select bands
VH = sentinel1.select('VH')
VV = sentinel1.select('VV')
slope = ee.Terrain.slope(SRTM)

# Function to normalize data using Min-Max scaling
def normalize(image):
    bandNames = image.bandNames()
    minDict = image.reduceRegion(reducer=ee.Reducer.min(), geometry=geometry, scale=scale, maxPixels=1e8, bestEffort=True, tileScale=4)
    maxDict = image.reduceRegion(reducer=ee.Reducer.max(), geometry=geometry, scale=scale, maxPixels=1e8, bestEffort=True, tileScale=4)
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))
    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized

In [232]:
# Combine all parameters
param = VH.addBands(VV).addBands(SRTM).addBands(slope).addBands(GLDAS)
normalizedImage = normalize(param)

# Prepare training data
GLDAS_Int = GLDAS.multiply(1000).toInt()
coorsfeat = GLDAS_Int.reduceToVectors(reducer=None, geometry=geometry, scale=27830, geometryType='centroid', labelProperty='GWS', eightConnected=False)

# Extract training data
training = normalizedImage.sampleRegions(collection=coorsfeat, properties=['GWS'], scale=27830, geometries=True)

In [233]:
# Split data into training and testing sets
training = training.randomColumn()
train = training.filter(ee.Filter.lt('random', 0.6))
test = training.filter(ee.Filter.gte('random', 0.6))

# Train a Random Forest regressor with fewer trees
bands = ['VH', 'VV', 'elevation', 'slope']
classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=n_tree,  
    variablesPerSplit=1,
    minLeafPopulation=1,
    bagFraction=0.5,
    maxNodes=4,
    seed=0
).setOutputMode('REGRESSION').train(
    features=train,
    classProperty='GWS',
    inputProperties=bands
)

# Predict using the trained model
regression = normalizedImage.select(bands).classify(classifier, 'predicted')
regression_Scaled = regression.divide(1000)

In [234]:
viz = {
    'min': regression_Scaled.reduceRegion(
        reducer=ee.Reducer.min(), 
        geometry=geometry, 
        scale=scale, 
        bestEffort=True, 
        maxPixels=1e8
    ).getInfo()['predicted'],  # Ensure 'predicted' is the correct key
    'max': regression_Scaled.reduceRegion(
        reducer=ee.Reducer.max(), 
        geometry=geometry, 
        scale=scale, 
        bestEffort=True, 
        maxPixels=1e8
    ).getInfo()['predicted'],  # Ensure 'predicted' is the correct key
    'palette': ['blue', 'green', 'red', 'orange']
}

In [235]:
# Create a map and add layers
Map = geemap.Map()
Map.addLayer(geometry, {}, 'Batas Kabupaten Ketapang')
Map.addLayer(GLDAS, colorizedVis, 'GWS')
Map.addLayer(VH, {'min': -38.8508, 'max': 21.1048, 'palette': viz['palette']}, 'VH')
Map.addLayer(VV, {'min': -32.3815, 'max': 26.9536, 'palette': viz['palette']}, 'VV')
Map.addLayer(SRTM, {'min': 0, 'max': 20300, 'palette': viz['palette']}, 'SRTM')
Map.addLayer(slope, {'min': 0, 'max': 83.6035, 'palette': viz['palette']}, 'Slope')
Map.addLayer(train, {'color': '0000FF'}, 'Train')
Map.addLayer(test, {'color': 'DC143C'}, 'Test')
Map.addLayer(coorsfeat, {'color': 'FF0000'}, 'Training Sample')
Map.addLayer(regression_Scaled, viz, 'Regression')

In [236]:
# Display the map
Map.addLayerControl()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…